In [ ]:
# =============================================================================
# STEP 6: EVENTSTREAM FUNCTIONALITY – REAL-TIME DATA INGESTION & STREAM PROCESSING
# =============================================================================
print("\n🌐 STEP 6: EVENTSTREAM FUNCTIONALITY – LIVE STREAM ANALYTICS")

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, BooleanType, TimestampType

# Initialize Spark Session
spark = SparkSession.builder.appName("ReimageAI_EventStream").getOrCreate()

# Define schema for incoming sensor events
event_schema = StructType([
    StructField("sensor_id", StringType(), True),
    StructField("parking_zone", StringType(), True),
    StructField("occupancy_status", BooleanType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("vibration_level", DoubleType(), True),
    StructField("blockchain_verified", BooleanType(), True),
    StructField("timestamp", TimestampType(), True)
])

# Read streaming data (example using Kafka-like Eventstream)
print("🔄 Listening for live events from Eventstream...")
eventstream_df = (
    spark.readStream
         .format("eventhubs")  # For Fabric Eventstream, replace with "eventhubs"
         .option("eventhubs.connectionString", "<your_eventhub_connection_string>")
         .load()
)

# Extract the body (JSON message)
parsed_df = eventstream_df.select(
    from_json(col("body").cast("string"), event_schema).alias("data")
).select("data.*")

# Enrich and save data
processed_df = parsed_df.withColumn("received_time", current_timestamp())

# Write to Delta table in near-real time
query = (
    processed_df.writeStream
        .format("delta")
        .option("checkpointLocation", "checkpoints/parking_eventstream_cp")
        .outputMode("append")
        .table("ParkingSensorData_Streaming")
)

print("✅ Eventstream processing started. Incoming events are being ingested live...")

# Monitor stream
spark.streams.awaitAnyTermination()
